In [6]:
import bagpy
import pandas as pd
import sys
import numpy as np
import os
from scipy.spatial.transform import Rotation as R

conversion = 0.18
file_path = '/home/shc/Desktop/data/0227/phacon_stable.bag'
b = bagpy.bagreader(file_path)
b.topic_table

[INFO]  Successfully created the data folder /home/shc/Desktop/data/0227/phacon_stable.


,Topics,Types,Message Count,Frequency
0,/fwd_pose_camhand,geometry_msgs/PoseStamped,122,29.827859
1,/fwd_pose_pan,geometry_msgs/PoseStamped,122,29.860349


In [7]:
pose_cam_data = b.message_by_topic('/fwd_pose_camhand')
pose_pan_data = b.message_by_topic('/fwd_pose_pan')

df_pan = pd.read_csv(pose_pan_data)[:499]
df_cam_hand = pd.read_csv(pose_cam_data)[:499]

In [8]:
os.chdir('/home/shc/Twin-S/util')
from Solver import solver
from dataLoader import dataLoader
sol = solver()
ld = dataLoader()

phacon2pan = np.load('/home/shc/Twin-S/params/phacon2pan_0227.npy')

X = np.load("/home/shc/Desktop/data/0220/handeye_1/hand_eye_X.npy")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [9]:
def getRealPose(idx, pd_data):
    df = pd_data
    pose_x = df['pose.position.x'][idx]*1000
    pose_y = df['pose.position.y'][idx]*1000
    pose_z = df['pose.position.z'][idx]*1000
    orin_x = df['pose.orientation.x'][idx]
    orin_y = df['pose.orientation.y'][idx]
    orin_z = df['pose.orientation.z'][idx]
    orin_w = df['pose.orientation.w'][idx]
    pose = np.array([pose_x, pose_y, pose_z, orin_x, orin_y, orin_z, orin_w])
    # print(real_pose, '\n', pose)
    return pose

In [10]:
num_frames = len(df_cam_hand)
pose_pan_data = np.zeros([num_frames, 4, 4])
pose_camhand_data = np.zeros([num_frames, 4, 4])
for i in range(num_frames):
    quaternion_pan = getRealPose(i, df_pan)
    quaternion_camhand = getRealPose(i, df_cam_hand)
    _, op2pan = sol.seven2trans(quaternion_pan)
    _, op2camhand = sol.seven2trans(quaternion_camhand)
    
    pan2phacon = sol.invTransformation(phacon2pan)
    op2phacon = op2pan@pan2phacon
    op2cam = op2camhand@X
     ## F_cv_ambf, cam2ambf
    extrinsic = np.array([[0, 1, 0, 0], [0, 0, -1, 0],
                          [-1, 0, 0, 0], [0, 0, 0, 1]])
    # op2cam = op2cam@extrinsic
    cam2phacon = sol.invTransformation(op2cam)@op2phacon
print(cam2phacon)
# r = R.from_matrix(cam2phacon[:3,:3])
# r.as_euler('xyz', degrees=True)


[[ 3.31579820e-01 -1.26244232e-01  9.34942360e-01  8.91062260e+00]
 [-7.35223071e-01 -6.55579920e-01  1.72226606e-01 -1.26785632e+01]
 [ 5.91186823e-01 -7.44498060e-01 -3.10194743e-01  1.62528537e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
# op2pan

In [12]:
# pan2phacon

In [13]:
# ambf2phacon = sol.invTransformation(extrinsic)@cam2phacon

In [14]:
# t = ambf2phacon[:3, 3]/conversion/1000
# r = R.from_matrix(ambf2phacon[:3,:3])
# Rot = r.as_euler('xyz')
# # Rot = Rot@Rot_offset
# print('position: {{x: {0}, y: {1}, z: {2}}}'.format(t[0],t[1],t[2]))
# print('orientation: {{r: {0}, p: {1}, y: {2}}}'.format(Rot[0], Rot[1], Rot[2]))

Fix the phacon to the AMBF origin

In [15]:
phacon2op = sol.invTransformation(op2pan@pan2phacon)
phacon2op

array([[ 8.64982409e-01,  3.09065129e-01, -3.95327938e-01,
         2.38287544e+02],
       [ 2.47677159e-01, -9.48119261e-01, -1.99313554e-01,
         2.60188342e+02],
       [-4.36418901e-01,  7.44890178e-02, -8.96654855e-01,
         1.01701033e+03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])